In [ ]:
from threading import *
import time 
from datetime import datetime
from time import sleep
from datetime import timedelta

In [ ]:
#import rpy2.robjects as robjects #r-python interfacing

import rpy2.robjects as robjects
from pymeasure.instruments import list_resources

import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
from ftd2xx import listDevices, getDeviceInfoDetail



from newportxps import NewportXPS
from newportxps.XPS_C8_drivers import XPS, XPSException


import numpy as np
import csv


xpsd_remoteip = '192.168.254.254'
# xpsd_hostip ='192.168.0.254'
hostname = 'XPS-1b81'

xps = NewportXPS(xpsd_remoteip)
print(xps.status_report())


xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()



def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        
        




# Initialize motion controller

In [ ]:

#pos_all() tells about current position of the motion controller

pos_all()


xps.move_stage('XYZ.Pos1',10) #moves x axis
xps.move_stage('XYZ.Pos2',11) #moves y axis
xps.move_stage('XYZ.Pos3',8.03) #moves z axis

In [ ]:
pos_all()

# Initialize Spectrometer

In [ ]:
from spectra import *
# from spectra import capture_photo,ration
capture_photo("start",2)

In [ ]:
mirror('on')

In [ ]:
st=7.99
zz=[float(st)+(0.02*i) for i in range(4)]
zz

In [ ]:
st=7.97

zz=[float(st)+(0.02*i) for i in range(6)]
for i in range(len(zz)):
    xps.move_stage('XYZ.Pos3',zz[i])
    print("z axis at: ",zz[i])
    time.sleep(8)


# Adjust the z-axis

In [ ]:
##calling the adjust() function will autofocus the z axis.

def adjust():
    line=1
    import timeit
#     begin=8.7
    start = timeit.default_timer()
    import time

    st=7.5

    zz=[float(st)+(0.1*i) for i in range(11)]
    intensity=[]
    for i in range(len(zz)):
        time.sleep(4)

        xps.move_stage('XYZ.Pos3',zz[i])
        time.sleep(1)
        p=capture_photo('adjust',i,line,1)
        intensity.append(p)
    mirror('off')
   # stop = timeit.default_timer()

    #print('Time: ', stop - start)  
    #print(intensity)

    import numpy as np
    ix=intensity.index(np.max(intensity))
    xps.move_stage('XYZ.Pos3',zz[ix])
    print("adjusted z position is: ",zz[ix])
    return zz[ix]

# Initialize Pressure


**current_pressure() reads the current pressure inside the chamber.** 

**open_valve('valve_name') is used to release gas from valve named as valve_name**

**valve_name= vac , out. vac stands for vaccum, out stands for outlet valve. so if you want to release the outlet then type --> open_valve('out')**

**To set the pressure inside the chamber, use gopr(pressure_value) function, suppose you need to set the pressure to 510 psi, then write the code --> gopr(510)**

**To close all the valves type -> close_all()**

**To close vac valve, type -> close_valve('vac')**

In [ ]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr

current_pressure()

# Initialize Laser

laser_power.power=0.01 # sets the power to 1 mW

In [ ]:
from pymeasure.instruments.lighthousephotonics import Sprout
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power=0.01

# Initialize Mirror

In [ ]:

from flipper import *

mirror("on")
# mirror("off")

In [ ]:
pos_all()

# Directory things

In [ ]:
import os
os.getcwd()
# os.mkdir('task1')
os.chdir(r'task1')

# Pattern a spot


In [ ]:
print("Enter the x,y position: ")

x,y=[x for x in input().split()]

print("Enter the power in mW")
power_i=int(input())

print("Enter the time of irradiation in ms")
time_i=int(input())

print("enter spot number")
i=int(input())

print("Enter the pressure in psi")
pressure_i=int(input())

for i in range(3):gopr(pressure_i)

In [ ]:
#move to the position where the patterning will take place
# xps.move_stage('XYZ.Pos1',x) #moves x axis
# xps.move_stage('XYZ.Pos2',y) #moves y axis
# xps.move_stage('XYZ.Pos3',z) #moves z axis


##check the spectra for precursor
# mirror("on")
# GD=capture_photo("before",i) #i is the spot number
# mirror("off")


####irradiation begins below
laser_power.power=((power_i-2.7979)/0.4639)/1000


time.sleep(25) #allow the power to reach it's value and get stabilized

mirror("on") #irradiation starts
time.sleep(time_i/1000) # mirror is open until irradiation period
mirror("off")
laser_power.power=0.01 # restore laser power to default



###now patterning is done. next step is to get the raman spectra

# mirror("on") 
# p=capture_photo("after",i)

# print("GD ratio is: ",p)

#For getting fitted curves, go to curve fit analysis lines and spots.ipynb file


# Next codes help to manually save the G/D ratio with the input parameters in a csv file

In [ ]:

i=2
p=capture_photo("after",i)

print("GD ratio is: ",p)

In [ ]:
df=pd.DataFrame(columns=['power','time','pressure','ratio'])
df['power'].loc[i]=power_i
df['time'].loc[i]=time_i
df['pressure'].loc[i]=120
df['ratio'].loc[i]=p

In [ ]:
df.to_csv('task1.csv')

In [ ]:
mirror("on") #irradiation starts
time.sleep(10000/1000) # mirror is open until irradiation period
mirror("off")
laser_power.power=0.01

In [ ]:
mirror('off')

In [ ]:
mirror('on')

In [ ]:
close_all()

# open_valve('out')

current_pressure()